<a href="https://colab.research.google.com/github/CeliaDiazSanchez/Data-Mining/blob/LAB01/LAB01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import requests
import pandas as pd

# URL del archivo CSV en GitHub
url = "https://raw.githubusercontent.com/CeliaDiazSanchez/Data-Mining/LAB01/vacunaciones.csv"

# Realizar la solicitud GET para obtener el contenido del archivo
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Guardar el contenido del archivo en un archivo local
    with open('vacunaciones.csv', 'wb') as f:
        f.write(response.content)
    print("Archivo descargado exitosamente.")
else:
    print("Error al descargar el archivo.")

# Cargar el archivo CSV en un DataFrame de pandas
df = pd.read_csv('vacunaciones.csv')

# Ahora puedes trabajar con el DataFrame df como desees
print(df.head(3))

Archivo descargado exitosamente.
   country iso_code        date  total_vaccinations  people_vaccinated  \
0  Albania      ALB  2021-01-10                 0.0                0.0   
1  Albania      ALB  2021-01-11                 NaN                NaN   
2  Albania      ALB  2021-01-12               128.0              128.0   

   people_fully_vaccinated  daily_vaccinations_raw  daily_vaccinations  \
0                      NaN                     NaN                 NaN   
1                      NaN                     NaN                64.0   
2                      NaN                     NaN                64.0   

   total_vaccinations_per_hundred  people_vaccinated_per_hundred  \
0                             0.0                            0.0   
1                             NaN                            NaN   
2                             0.0                            0.0   

   people_fully_vaccinated_per_hundred  daily_vaccinations_per_million  \
0                         